In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import CubicSpline


In [2]:
df = pd.read_csv("Br_vegetation_with_City.csv")
df.head()

,date,name,geocode,vim,vim_avg,viq
0,2002-07-01,Nilópolis,3303203,0.4761,0.4963,96.3034
1,2002-07-11,Nilópolis,3303203,0.4692,0.4844,97.1523
2,2002-07-21,Nilópolis,3303203,0.4605,0.4710,97.9752
3,2002-08-01,Nilópolis,3303203,0.4511,0.4589,98.4673
4,2002-08-11,Nilópolis,3303203,0.4422,0.4493,98.5854


In [3]:
df.isnull().sum()


date       0
name       0
geocode    0
vim        0
vim_avg    0
viq        0
dtype: int64

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72709 entries, 0 to 72708
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   date     72709 non-null  object 
 1   name     72709 non-null  object 
 2   geocode  72709 non-null  int64  
 3   vim      72709 non-null  float64
 4   vim_avg  72709 non-null  float64
 5   viq      72709 non-null  float64
dtypes: float64(3), int64(1), object(2)
memory usage: 3.3+ MB


In [5]:
# Filter rows where the date is on or after '2012-01-01'
df = df[
    (df['date'] >= '2012-01-01') & 
    (df['date'] <= '2022-12-31')
]

# Display the filtered dataset inf
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36036 entries, 342 to 72647
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   date     36036 non-null  object 
 1   name     36036 non-null  object 
 2   geocode  36036 non-null  int64  
 3   vim      36036 non-null  float64
 4   vim_avg  36036 non-null  float64
 5   viq      36036 non-null  float64
dtypes: float64(3), int64(1), object(2)
memory usage: 1.9+ MB


MONTHLY INTERPOLATION

In [6]:
# Ensure 'date' column is of datetime type
df['date'] = pd.to_datetime(df['date'])

# Extract year and month from the date for monthly aggregation
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

# Monthly aggregation: Calculate monthly mean of 'vim', 'vim_avg', 'viq'
monthly_vegetation = df.groupby(['geocode', 'year', 'month']).agg({
    'vim': 'mean',
    'vim_avg': 'mean',
    'viq': 'mean'
}).reset_index()

monthly_vegetation.head()

,geocode,year,month,vim,vim_avg,viq
0,3300100,2012,1,0.852633,0.857233,99.495800
1,3300100,2012,2,0.858000,0.864000,99.344433
2,3300100,2012,3,0.861567,0.867967,99.302833
3,3300100,2012,4,0.863667,0.868800,99.441267
4,3300100,2012,5,0.863833,0.865667,99.801567


In [7]:
monthly_vegetation = monthly_vegetation.rename(columns={'vim': 'vim_monthly'})
monthly_vegetation.head(15)


,geocode,year,month,vim_monthly,vim_avg,viq
0,3300100,2012,1,0.852633,0.857233,99.495800
1,3300100,2012,2,0.858000,0.864000,99.344433
2,3300100,2012,3,0.861567,0.867967,99.302833
3,3300100,2012,4,0.863667,0.868800,99.441267
4,3300100,2012,5,0.863833,0.865667,99.801567
5,3300100,2012,6,0.861067,0.859100,100.217967
6,3300100,2012,7,0.854900,0.850333,100.510467
7,3300100,2012,8,0.847200,0.842567,100.517900
8,3300100,2012,9,0.841933,0.837800,100.467033
9,3300100,2012,10,0.842233,0.837500,100.532267


WEEKLY INTERPOLATION USING CUBIC SPLINE

In [8]:
df = df.sort_values(by=['geocode', 'date'])
print(df)

            date            name  geocode     vim  vim_avg       viq  month  \
36297 2012-01-01  Angra dos Reis  3300100  0.8505   0.8544   99.5724      1   
36298 2012-01-11  Angra dos Reis  3300100  0.8527   0.8574   99.4879      1   
36299 2012-01-21  Angra dos Reis  3300100  0.8547   0.8599   99.4271      1   
36300 2012-02-01  Angra dos Reis  3300100  0.8565   0.8622   99.3780      2   
36301 2012-02-11  Angra dos Reis  3300100  0.8581   0.8641   99.3404      2   
...          ...             ...      ...     ...      ...       ...    ...   
70246 2022-11-11   Volta Redonda  3306305  0.6422   0.6328  101.3823     11   
70247 2022-11-21   Volta Redonda  3306305  0.6704   0.6526  102.5213     11   
70248 2022-12-01   Volta Redonda  3306305  0.6966   0.6692  103.8161     12   
70249 2022-12-11   Volta Redonda  3306305  0.7191   0.6817  105.1096     12   
70250 2022-12-21   Volta Redonda  3306305  0.7350   0.6904  106.0223     12   

       year  
36297  2012  
36298  2012  
36299  20

In [9]:

# Ensure the 'date' column is in datetime format
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by=['geocode', 'date'])

# Define the function for spline interpolation on 'vim' for each geo code
def interpolate_vim_spline(group):
    # Extract year for the group
    group['year'] = group['date'].dt.year
    
    # Define an empty DataFrame to store the results
    interpolated_df = pd.DataFrame()

    # Loop through each year in the group and interpolate separately
    for year in group['year'].unique():
        year_group = group[group['year'] == year]
        
        # Convert date to ordinal (number of days since 1970-01-01)
        year_group['date_ordinal'] = (year_group['date'] - pd.Timestamp('1970-01-01')).dt.days
        
        # Create a cubic spline interpolator for 'vim' values
        spline = CubicSpline(year_group['date_ordinal'], year_group['vim'], bc_type='natural')
        
        # Generate weekly date intervals for the year
        weekly_dates = pd.date_range(start=f'{year}-01-01', end=f'{year}-12-31', freq='7D')
        
        # Convert weekly dates to ordinal (days since 1970-01-01)
        weekly_ordinals = (weekly_dates - pd.Timestamp('1970-01-01')).days
        
        # Interpolate the 'vim' values for the weekly dates
        weekly_vim = spline(weekly_ordinals)
        
        # Calculate custom year-week values
        start_of_year = pd.Timestamp(f'{year}-01-01')
        weekly_year_week = ((weekly_dates - start_of_year).days // 7 + 1).astype(int).astype(str)
        weekly_year_week = weekly_dates.year.astype(str) + weekly_year_week.str.zfill(2)
        
        # Create a DataFrame for the interpolated values for the year
        year_interpolated_df = pd.DataFrame({
            'date': weekly_dates,
            'vim': weekly_vim,
            'geocode': year_group['geocode'].iloc[0],  # Use the first geocode of the year group
            'week': weekly_year_week  # Add the custom year-week column
        })
        
        # Append the result for this year to the overall DataFrame
        interpolated_df = pd.concat([interpolated_df, year_interpolated_df], ignore_index=True)
    
    return interpolated_df

# Apply the spline interpolation function for each geo code
df_interpolated_spline = df.groupby('geocode').apply(interpolate_vim_spline).reset_index(drop=True)

df_interpolated_spline.head(65)


C:\Users\user\AppData\Local\Temp\ipykernel_4232\328760175.py:51: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_interpolated_spline = df.groupby('geocode').apply(interpolate_vim_spline).reset_index(drop=True)


,date,vim,geocode,week
0,2012-01-01,0.850500,3300100,201201
1,2012-01-08,0.852050,3300100,201202
2,2012-01-15,0.853541,3300100,201203
3,2012-01-22,0.854877,3300100,201204
4,2012-01-29,0.856021,3300100,201205
...,...,...,...,...
60,2013-02-19,0.873353,3300100,201308
61,2013-02-26,0.874238,3300100,201309
62,2013-03-05,0.875015,3300100,201310
63,2013-03-12,0.875567,3300100,201311


In [10]:
df_interpolated_spline.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53053 entries, 0 to 53052
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     53053 non-null  datetime64[ns]
 1   vim      53053 non-null  float64       
 2   geocode  53053 non-null  int64         
 3   week     53053 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 1.6+ MB


In [11]:
df_interpolated_spline.nunique()

date         583
vim        50698
geocode       91
week         583
dtype: int64

In [12]:
dengue_df = pd.read_csv("52_week_dengue_dataset.csv")
dengue_df = dengue_df.sort_values(by=['geo_code', 'date'])
dengue_df.head()

,date,week,cases,population,tempe_min,humidity_max,humidity_avg,humidity_min,temp_avg,temp_max,city,geo_code
51,2012-01-01,201201,32,207044.0,19.000000,79.428571,55.514486,35.000000,25.048951,29.571429,Angra dos Reis,3300100
50,2012-01-08,201202,40,207044.0,19.714286,82.285714,62.357393,47.428571,23.737513,26.571429,Angra dos Reis,3300100
49,2012-01-15,201203,19,207044.0,20.000000,83.000000,65.236264,45.571429,24.413187,28.714286,Angra dos Reis,3300100
48,2012-01-22,201204,33,207044.0,19.285714,83.000000,60.362637,43.428571,24.879121,28.857143,Angra dos Reis,3300100
47,2012-01-29,201205,36,207044.0,18.857143,80.857143,50.885924,33.142857,25.989992,30.428571,Angra dos Reis,3300100


In [13]:
dengue_df.tail()

,date,week,cases,population,tempe_min,humidity_max,humidity_avg,humidity_min,temp_avg,temp_max,city,geo_code
52760,2022-11-27,202248,1,273988.0,21.857143,95.875069,82.840922,61.313170,25.507143,30.857143,Volta Redonda,3306305
52759,2022-12-04,202249,0,273988.0,21.857143,92.543531,80.151192,64.110460,24.903139,28.142857,Volta Redonda,3306305
52758,2022-12-11,202250,2,273988.0,21.000000,94.213747,79.480439,59.964030,24.597222,28.500000,Volta Redonda,3306305
52757,2022-12-18,202251,1,273988.0,21.000000,91.312065,84.229947,73.950180,23.184295,26.000000,Volta Redonda,3306305
52756,2022-12-25,202252,0,273988.0,22.333333,92.282320,83.456510,71.528758,24.840404,27.666667,Volta Redonda,3306305


In [14]:
dengue_df = dengue_df.rename(columns={'geo_code': 'geocode'})
dengue_df.head()

,date,week,cases,population,tempe_min,humidity_max,humidity_avg,humidity_min,temp_avg,temp_max,city,geocode
51,2012-01-01,201201,32,207044.0,19.000000,79.428571,55.514486,35.000000,25.048951,29.571429,Angra dos Reis,3300100
50,2012-01-08,201202,40,207044.0,19.714286,82.285714,62.357393,47.428571,23.737513,26.571429,Angra dos Reis,3300100
49,2012-01-15,201203,19,207044.0,20.000000,83.000000,65.236264,45.571429,24.413187,28.714286,Angra dos Reis,3300100
48,2012-01-22,201204,33,207044.0,19.285714,83.000000,60.362637,43.428571,24.879121,28.857143,Angra dos Reis,3300100
47,2012-01-29,201205,36,207044.0,18.857143,80.857143,50.885924,33.142857,25.989992,30.428571,Angra dos Reis,3300100


In [15]:
dengue_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 52808 entries, 51 to 52756
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          52808 non-null  object 
 1   week          52808 non-null  int64  
 2   cases         52808 non-null  int64  
 3   population    52808 non-null  float64
 4   tempe_min     52808 non-null  float64
 5   humidity_max  50828 non-null  float64
 6   humidity_avg  50828 non-null  float64
 7   humidity_min  50828 non-null  float64
 8   temp_avg      50828 non-null  float64
 9   temp_max      50828 non-null  float64
 10  city          52808 non-null  object 
 11  geocode       52808 non-null  int64  
dtypes: float64(7), int64(3), object(2)
memory usage: 5.2+ MB


In [16]:
df_interpolated_spline.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53053 entries, 0 to 53052
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     53053 non-null  datetime64[ns]
 1   vim      53053 non-null  float64       
 2   geocode  53053 non-null  int64         
 3   week     53053 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 1.6+ MB


In [17]:
# Convert 'week' in dengue_df to string to match the type in df_interpolated_spline
dengue_df['week'] = dengue_df['week'].astype(str)

# Ensure 'week' in df_interpolated_spline is a string (already is, but for safety)
df_interpolated_spline['week'] = df_interpolated_spline['week'].astype(str)

# Perform the merge
dengue_df_merged = pd.merge(
    dengue_df,
    df_interpolated_spline[['geocode', 'week', 'vim']],  # Select relevant columns from interpolated DataFrame
    on=['geocode', 'week'],  # Merge on 'geocode' and 'week'
    how='left'  # Use a left join to retain all rows from dengue_df
)

# Display the merged DataFrame
dengue_df_merged.head()

# Display the shape of the merged DataFrame


,date,week,cases,population,tempe_min,humidity_max,humidity_avg,humidity_min,temp_avg,temp_max,city,geocode,vim
0,2012-01-01,201201,32,207044.0,19.000000,79.428571,55.514486,35.000000,25.048951,29.571429,Angra dos Reis,3300100,0.850500
1,2012-01-08,201202,40,207044.0,19.714286,82.285714,62.357393,47.428571,23.737513,26.571429,Angra dos Reis,3300100,0.852050
2,2012-01-15,201203,19,207044.0,20.000000,83.000000,65.236264,45.571429,24.413187,28.714286,Angra dos Reis,3300100,0.853541
3,2012-01-22,201204,33,207044.0,19.285714,83.000000,60.362637,43.428571,24.879121,28.857143,Angra dos Reis,3300100,0.854877
4,2012-01-29,201205,36,207044.0,18.857143,80.857143,50.885924,33.142857,25.989992,30.428571,Angra dos Reis,3300100,0.856021


In [18]:
dengue_df_merged.tail()


,date,week,cases,population,tempe_min,humidity_max,humidity_avg,humidity_min,temp_avg,temp_max,city,geocode,vim
52803,2022-11-27,202248,1,273988.0,21.857143,95.875069,82.840922,61.313170,25.507143,30.857143,Volta Redonda,3306305,0.683798
52804,2022-12-04,202249,0,273988.0,21.857143,92.543531,80.151192,64.110460,24.903139,28.142857,Volta Redonda,3306305,0.701526
52805,2022-12-11,202250,2,273988.0,21.000000,94.213747,79.480439,59.964030,24.597222,28.500000,Volta Redonda,3306305,0.717158
52806,2022-12-18,202251,1,273988.0,21.000000,91.312065,84.229947,73.950180,23.184295,26.000000,Volta Redonda,3306305,0.729156
52807,2022-12-25,202252,0,273988.0,22.333333,92.282320,83.456510,71.528758,24.840404,27.666667,Volta Redonda,3306305,0.739351


In [19]:
dengue_df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52808 entries, 0 to 52807
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          52808 non-null  object 
 1   week          52808 non-null  object 
 2   cases         52808 non-null  int64  
 3   population    52808 non-null  float64
 4   tempe_min     52808 non-null  float64
 5   humidity_max  50828 non-null  float64
 6   humidity_avg  50828 non-null  float64
 7   humidity_min  50828 non-null  float64
 8   temp_avg      50828 non-null  float64
 9   temp_max      50828 non-null  float64
 10  city          52808 non-null  object 
 11  geocode       52808 non-null  int64  
 12  vim           52234 non-null  float64
dtypes: float64(8), int64(2), object(3)
memory usage: 5.2+ MB


In [20]:
dengue_df_merged.isnull().sum()

date               0
week               0
cases              0
population         0
tempe_min          0
humidity_max    1980
humidity_avg    1980
humidity_min    1980
temp_avg        1980
temp_max        1980
city               0
geocode            0
vim              574
dtype: int64

In [21]:
#drop all the columns of mesquita
# Remove rows where the 'city' is equal to 'Mesquita'
dengue_df_merged = dengue_df_merged[dengue_df_merged['city'] != 'Mesquita']

# Verify the removal
print(dengue_df_merged['city'].nunique())

91


In [22]:
dengue_df_merged.isnull().sum()

date               0
week               0
cases              0
population         0
tempe_min          0
humidity_max    1957
humidity_avg    1957
humidity_min    1957
temp_avg        1957
temp_max        1957
city               0
geocode            0
vim                0
dtype: int64

In [23]:

dengue_df_merged['date'] = pd.to_datetime(dengue_df_merged['date'])

dengue_df_merged['year'] = dengue_df_merged['date'].dt.year
dengue_df_merged['month'] = dengue_df_merged['date'].dt.month
dengue_df_merged.head(10)

,date,week,cases,population,tempe_min,humidity_max,humidity_avg,humidity_min,temp_avg,temp_max,city,geocode,vim,year,month
0,2012-01-01,201201,32,207044.0,19.000000,79.428571,55.514486,35.000000,25.048951,29.571429,Angra dos Reis,3300100,0.850500,2012,1
1,2012-01-08,201202,40,207044.0,19.714286,82.285714,62.357393,47.428571,23.737513,26.571429,Angra dos Reis,3300100,0.852050,2012,1
2,2012-01-15,201203,19,207044.0,20.000000,83.000000,65.236264,45.571429,24.413187,28.714286,Angra dos Reis,3300100,0.853541,2012,1
3,2012-01-22,201204,33,207044.0,19.285714,83.000000,60.362637,43.428571,24.879121,28.857143,Angra dos Reis,3300100,0.854877,2012,1
4,2012-01-29,201205,36,207044.0,18.857143,80.857143,50.885924,33.142857,25.989992,30.428571,Angra dos Reis,3300100,0.856021,2012,1
5,2012-02-05,201206,39,207044.0,19.571429,81.571429,43.494048,22.142857,28.614927,33.571429,Angra dos Reis,3300100,0.857159,2012,2
6,2012-02-12,201207,37,207044.0,20.142857,80.142857,56.778911,40.571429,25.791314,29.571429,Angra dos Reis,3300100,0.858241,2012,2
7,2012-02-19,201208,33,207044.0,21.857143,74.000000,51.458425,34.428571,27.253296,31.285714,Angra dos Reis,3300100,0.859142,2012,2
8,2012-02-26,201209,46,207044.0,20.714286,80.142857,43.760989,21.285714,28.902930,34.000000,Angra dos Reis,3300100,0.860082,2012,2
9,2012-03-04,201210,42,207044.0,20.428571,71.714286,48.333639,28.428571,26.467491,30.857143,Angra dos Reis,3300100,0.860933,2012,3


In [24]:
monthly_vegetation.head(10)

,geocode,year,month,vim_monthly,vim_avg,viq
0,3300100,2012,1,0.852633,0.857233,99.495800
1,3300100,2012,2,0.858000,0.864000,99.344433
2,3300100,2012,3,0.861567,0.867967,99.302833
3,3300100,2012,4,0.863667,0.868800,99.441267
4,3300100,2012,5,0.863833,0.865667,99.801567
5,3300100,2012,6,0.861067,0.859100,100.217967
6,3300100,2012,7,0.854900,0.850333,100.510467
7,3300100,2012,8,0.847200,0.842567,100.517900
8,3300100,2012,9,0.841933,0.837800,100.467033
9,3300100,2012,10,0.842233,0.837500,100.532267


In [25]:
monthly_vegetation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12012 entries, 0 to 12011
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   geocode      12012 non-null  int64  
 1   year         12012 non-null  int32  
 2   month        12012 non-null  int32  
 3   vim_monthly  12012 non-null  float64
 4   vim_avg      12012 non-null  float64
 5   viq          12012 non-null  float64
dtypes: float64(3), int32(2), int64(1)
memory usage: 469.3 KB


In [26]:
dengue_df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 52234 entries, 0 to 52807
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          52234 non-null  datetime64[ns]
 1   week          52234 non-null  object        
 2   cases         52234 non-null  int64         
 3   population    52234 non-null  float64       
 4   tempe_min     52234 non-null  float64       
 5   humidity_max  50277 non-null  float64       
 6   humidity_avg  50277 non-null  float64       
 7   humidity_min  50277 non-null  float64       
 8   temp_avg      50277 non-null  float64       
 9   temp_max      50277 non-null  float64       
 10  city          52234 non-null  object        
 11  geocode       52234 non-null  int64         
 12  vim           52234 non-null  float64       
 13  year          52234 non-null  int32         
 14  month         52234 non-null  int32         
dtypes: datetime64[ns](1), float64(8), int32(2

In [27]:

dengue_df_merged = pd.merge(dengue_df_merged,
                            monthly_vegetation[['geocode', 'year', 'month', 'vim_monthly']], 
                            on=['geocode', 'year', 'month'], 
                            how='left')

dengue_df_merged.head(10)

,date,week,cases,population,tempe_min,humidity_max,humidity_avg,humidity_min,temp_avg,temp_max,city,geocode,vim,year,month,vim_monthly
0,2012-01-01,201201,32,207044.0,19.000000,79.428571,55.514486,35.000000,25.048951,29.571429,Angra dos Reis,3300100,0.850500,2012,1,0.852633
1,2012-01-08,201202,40,207044.0,19.714286,82.285714,62.357393,47.428571,23.737513,26.571429,Angra dos Reis,3300100,0.852050,2012,1,0.852633
2,2012-01-15,201203,19,207044.0,20.000000,83.000000,65.236264,45.571429,24.413187,28.714286,Angra dos Reis,3300100,0.853541,2012,1,0.852633
3,2012-01-22,201204,33,207044.0,19.285714,83.000000,60.362637,43.428571,24.879121,28.857143,Angra dos Reis,3300100,0.854877,2012,1,0.852633
4,2012-01-29,201205,36,207044.0,18.857143,80.857143,50.885924,33.142857,25.989992,30.428571,Angra dos Reis,3300100,0.856021,2012,1,0.852633
5,2012-02-05,201206,39,207044.0,19.571429,81.571429,43.494048,22.142857,28.614927,33.571429,Angra dos Reis,3300100,0.857159,2012,2,0.858000
6,2012-02-12,201207,37,207044.0,20.142857,80.142857,56.778911,40.571429,25.791314,29.571429,Angra dos Reis,3300100,0.858241,2012,2,0.858000
7,2012-02-19,201208,33,207044.0,21.857143,74.000000,51.458425,34.428571,27.253296,31.285714,Angra dos Reis,3300100,0.859142,2012,2,0.858000
8,2012-02-26,201209,46,207044.0,20.714286,80.142857,43.760989,21.285714,28.902930,34.000000,Angra dos Reis,3300100,0.860082,2012,2,0.858000
9,2012-03-04,201210,42,207044.0,20.428571,71.714286,48.333639,28.428571,26.467491,30.857143,Angra dos Reis,3300100,0.860933,2012,3,0.861567


In [28]:
dengue_df_merged.isnull().sum()

date               0
week               0
cases              0
population         0
tempe_min          0
humidity_max    1957
humidity_avg    1957
humidity_min    1957
temp_avg        1957
temp_max        1957
city               0
geocode            0
vim                0
year               0
month              0
vim_monthly        0
dtype: int64

In [29]:
dengue_df_merged = dengue_df_merged.drop(columns=['year', 'month'])



In [30]:
dengue_df_merged.head(10)

,date,week,cases,population,tempe_min,humidity_max,humidity_avg,humidity_min,temp_avg,temp_max,city,geocode,vim,vim_monthly
0,2012-01-01,201201,32,207044.0,19.000000,79.428571,55.514486,35.000000,25.048951,29.571429,Angra dos Reis,3300100,0.850500,0.852633
1,2012-01-08,201202,40,207044.0,19.714286,82.285714,62.357393,47.428571,23.737513,26.571429,Angra dos Reis,3300100,0.852050,0.852633
2,2012-01-15,201203,19,207044.0,20.000000,83.000000,65.236264,45.571429,24.413187,28.714286,Angra dos Reis,3300100,0.853541,0.852633
3,2012-01-22,201204,33,207044.0,19.285714,83.000000,60.362637,43.428571,24.879121,28.857143,Angra dos Reis,3300100,0.854877,0.852633
4,2012-01-29,201205,36,207044.0,18.857143,80.857143,50.885924,33.142857,25.989992,30.428571,Angra dos Reis,3300100,0.856021,0.852633
5,2012-02-05,201206,39,207044.0,19.571429,81.571429,43.494048,22.142857,28.614927,33.571429,Angra dos Reis,3300100,0.857159,0.858000
6,2012-02-12,201207,37,207044.0,20.142857,80.142857,56.778911,40.571429,25.791314,29.571429,Angra dos Reis,3300100,0.858241,0.858000
7,2012-02-19,201208,33,207044.0,21.857143,74.000000,51.458425,34.428571,27.253296,31.285714,Angra dos Reis,3300100,0.859142,0.858000
8,2012-02-26,201209,46,207044.0,20.714286,80.142857,43.760989,21.285714,28.902930,34.000000,Angra dos Reis,3300100,0.860082,0.858000
9,2012-03-04,201210,42,207044.0,20.428571,71.714286,48.333639,28.428571,26.467491,30.857143,Angra dos Reis,3300100,0.860933,0.861567
